# Spider 데이터셋(translated korean question)에 대한 Llama 3.1 원본 모델 성능 테스트

In [1]:
# Google Drive 연결
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정 (Spider 데이터셋이 저장된 경로로 수정하세요)
SPIDER_DIR = '/content/drive/MyDrive/spider/spider_data'  # 필요에 따라 수정
DATASET_PATH = '/content/drive/MyDrive/spider/spider_data/spider_train_translations_final.csv'  # 필요에 따라 수정

Mounted at /content/drive


In [2]:

!pip uninstall unsloth triton -y
#!pip install triton vllm unsloth tqdm pandas bitsandbytes
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes


Found existing installation: triton 3.2.0
Uninstalling triton-3.2.0:
  Successfully uninstalled triton-3.2.0
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-cmrtfzur/unsloth_e6069b52cbe1404a898086bb5f2950b1
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-cmrtfzur/unsloth_e6069b52cbe1404a898086bb5f2950b1
  Resolved https://github.com/unslothai/unsloth.git to commit c1b73fa8836aa7e8b9ee13d748369f8f61e1fac5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 28.4 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.18.1
    Uninstalling trl-0.18.1:
      Successfully uninstalled trl-0.18.1


## 3. 모델 및 토크나이저 로드

In [4]:
from unsloth import FastLanguageModel
import torch

# 모델 구성
max_seq_length = 2048  # 필요에 따라 조정
dtype = None  # 자동 감지
load_in_4bit = True
model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"

# 모델 및 토크나이저 로드
print("Loading model and tokenizer...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.9 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
Loading model and tokenizer...
==((====))==  Unsloth 2025.6.1: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2025.6.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [20]:

# ==============================================================================
# 1. 라이브러리 임포트
# ==============================================================================
import json
import os
import sqlite3
import re
import random
from tqdm import tqdm
import pandas as pd
import torch
from unsloth import FastLanguageModel
from IPython.display import display

# ==============================================================================
# 2. 경로 및 기본 설정
# ==============================================================================
# !!! 중요: 이 경로를 실제 Spider 데이터셋이 있는 디렉토리로 변경하세요 !!!
SPIDER_DIR = '/content/drive/MyDrive/spider/spider_data'  # 예: '/content/spider' 또는 'data/spider'

# 사용할 모델의 Hugging Face 경로
# 예: "unsloth/llama-3.1-8b-Instruct-bnb-4bit"
MODEL_NAME = "unsloth/llama-3.1-8b-Instruct-bnb-4bit"


# ==============================================================================
# 3. 데이터 로딩 및 스키마 처리 헬퍼 함수
# ==============================================================================
def load_tables_data(tables_file):
    """tables.json 파일에서 테이블 스키마 정보를 로드합니다."""
    if not os.path.exists(tables_file):
        print(f"오류: '{tables_file}' 파일을 찾을 수 없습니다!")
        return None
    with open(tables_file, 'r', encoding='utf-8') as f:
        return json.load(f)

def create_schema_mapping(tables_data):
    """db_id를 키로 하는 스키마 정보 딕셔너리를 생성합니다."""
    schema_mapping = {}
    for item in tables_data:
        db_id = item["db_id"]
        tables = item["table_names_original"]
        columns = {tbl: [] for tbl in tables}
        column_types = {tbl: {} for tbl in tables}

        for i, (table_id, col_name) in enumerate(item["column_names_original"]):
            if table_id >= 0:
                table_name = tables[table_id]
                columns[table_name].append(col_name)
                column_types[table_name][col_name] = item["column_types"][i]

        primary_keys = []
        for pk_col_idx in item.get("primary_keys", []):
            table_id, col_name = item["column_names_original"][pk_col_idx]
            if table_id >= 0:
                primary_keys.append(f"{tables[table_id]}.{col_name}")

        schema_mapping[db_id] = {
            "tables": tables,
            "columns": columns,
            "column_types": column_types,
            "primary_keys": primary_keys,
        }
    return schema_mapping

def get_schema_text(db_id, schema_mapping):
    """주어진 db_id에 대한 텍스트 형식의 스키마를 생성합니다."""
    if db_id not in schema_mapping:
        return "Schema information not available"

    schema = schema_mapping[db_id]
    schema_text_parts = []

    for table_name in schema["tables"]:
        schema_text_parts.append(f"테이블: {table_name}")
        cols_info = []
        for col_name in schema["columns"].get(table_name, []):
            col_type = schema["column_types"][table_name][col_name]
            pk_marker = " (기본 키)" if f"{table_name}.{col_name}" in schema["primary_keys"] else ""
            cols_info.append(f"  - {col_name} ({col_type}){pk_marker}")
        schema_text_parts.append("컬럼:\n" + "\n".join(cols_info))

    return "\n".join(schema_text_parts)

# ==============================================================================
# 4. 핵심 로직: SQL 생성 함수
# ==============================================================================
def generate_sql(model, tokenizer, schema_info, question_ko):
    """
    모델에게 Markdown 코드 블록으로 응답하도록 지시하고, 후처리를 통해 SQL만 추출합니다.
    """
    messages = [
        {
            "role": "system",
            "content": "당신은 Text-to-SQL 전문가입니다. 주어진 스키마와 질문에 대한 SQL 쿼리를 생성하세요. 응답은 반드시 ```sql 코드 블록 안에 넣어야 합니다. 다른 어떤 설명도 추가하지 마세요."
        },
        {
            "role": "user",
            "content": f"아래 스키마와 질문을 바탕으로 SQL 쿼리를 markdown 코드 블록으로 작성해줘.\n\n[스키마]:\n{schema_info}\n\n[질문]:\n{question_ko}\n\n[SQL 쿼리]:"
        }
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        inputs,
        max_new_tokens=150,
        temperature=0.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    full_response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

    # --- 후처리 로직: ```sql 블록에서 SQL 쿼리 추출 ---
    match = re.search(r'```sql\s*(.*?)\s*```', full_response, re.DOTALL)

    if match:
        sql_query = match.group(1).strip()
        return sql_query
    else:
        # 모델이 코드 블록을 생성하지 않은 경우에 대한 대체(Fallback) 로직
        fallback_match = re.search(r'(?i)(SELECT|WITH|\(SELECT)', full_response)
        if fallback_match:
            return full_response[fallback_match.start():].strip()
        return full_response.strip() # 최악의 경우, 디버깅을 위해 그대로 반환

# ==============================================================================
# 5. 평가 헬퍼 함수
# ==============================================================================
def normalize_sql(sql):
    """SQL 쿼리를 비교하기 쉽도록 정규화합니다."""
    if not isinstance(sql, str):
        return ""
    sql = sql.lower().strip()
    sql = re.sub(r'\s+', ' ', sql)
    sql = re.sub(r'\s*,\s*', ', ', sql)
    sql = re.sub(r'\s*=\s*', ' = ', sql)
    sql = re.sub(r';+\s*$', '', sql)
    return sql

def execute_sql(db_path, sql):
    """SQLite 데이터베이스에서 SQL을 실행하고 결과를 반환합니다."""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(sql)
        results = cursor.fetchall()
        conn.close()
        return True, results
    except Exception as e:
        return False, str(e)

# ==============================================================================
# 6. 메인 평가 실행 함수
# ==============================================================================
def run_evaluation_ko_from_csv(model, tokenizer, n_samples=10, random_seed=42):
    """
    CSV 파일에서 한국어 질문을 사용하여 모델을 평가합니다.
    """
    try:
        csv_path = os.path.join(SPIDER_DIR, 'spider_validation_final.csv')
        tables_file = os.path.join(SPIDER_DIR, 'tables.json')
        db_dir = os.path.join(SPIDER_DIR, 'database')

        # 파일 존재 확인
        for path in [csv_path, tables_file, db_dir]:
            if not os.path.exists(path):
                print(f"❌ 필수 파일/디렉토리를 찾을 수 없습니다: {path}")
                print(f"SPIDER_DIR ('{SPIDER_DIR}') 경로가 올바른지 확인하세요.")
                return

        print(f"✅ 데이터셋 파일 확인 완료.")
        print(f"Loading CSV file from {csv_path}…")
        df = pd.read_csv(csv_path)

        # 스키마 정보 로드
        tables_data = load_tables_data(tables_file)
        schema_dict = create_schema_mapping(tables_data)

        # 랜덤 샘플 선택
        random.seed(random_seed)
        sample_indices = random.sample(range(len(df)), min(n_samples, len(df)))
        sampled_df = df.iloc[sample_indices]

        print(f"\n🚀 총 {len(sampled_df)}개 랜덤 샘플에 대한 평가를 시작합니다…\n" + "="*80)

        results = []

        for i, (idx, row) in enumerate(sampled_df.iterrows()):
            db_id = row['db_id']
            question_ko = row['question_ko']
            gold_sql = row['query']

            print(f"\nExample {i+1}/{len(sampled_df)}:")
            print(f"Database: {db_id}")
            print(f"Question (KO): {question_ko}")
            print(f"Gold SQL: {gold_sql}")

            schema_info = get_schema_text(db_id, schema_dict)
            pred_sql = generate_sql(model, tokenizer, schema_info, question_ko)

            print(f"Generated SQL: {pred_sql}")

            # 정확한 일치(Exact Match) 평가
            exact_match = normalize_sql(gold_sql) == normalize_sql(pred_sql)
            print(f"Exact Match: {'✅' if exact_match else '❌'}")

            # 실행 정확도(Execution Accuracy) 평가
            db_path = os.path.join(db_dir, db_id, f"{db_id}.sqlite")
            execution_match = False
            execution_error = None

            if os.path.exists(db_path):
                gold_success, gold_result = execute_sql(db_path, gold_sql)
                pred_success, pred_result = execute_sql(db_path, pred_sql)

                if not pred_success:
                    execution_error = pred_result
                    print(f"Execution: ❌ (Error: {str(execution_error)[:100]}...)")
                elif not gold_success:
                     print(f"Execution: ⚠️ (Gold SQL 실행 오류)")
                else:
                    execution_match = (set(gold_result) == set(pred_result))
                    print(f"Execution Match: {'✅' if execution_match else '❌'}")
            else:
                execution_error = "DB file not found"
                print(f"Execution: ⚠️ (DB 파일을 찾을 수 없음)")

            print("-" * 80)

            results.append({
                'index': idx, 'db_id': db_id, 'question_ko': question_ko,
                'gold_sql': gold_sql, 'generated_sql': pred_sql,
                'exact_match': exact_match, 'execution_match': execution_match,
                'execution_error': execution_error
            })

        # 최종 결과 요약
        total = len(results)
        em_count = sum(r['exact_match'] for r in results)
        ex_count = sum(r['execution_match'] for r in results)

        print(f"\n{'='*80}\n📊 최종 평가 요약 (총 {total}개 예제):\n{'='*80}")
        print(f"  - 완전 일치 (EM): {em_count} / {total} ({em_count/total:.2%})")
        print(f"  - 실행 정확도 (EX): {ex_count} / {total} ({ex_count/total:.2%})")
        print(f"{'='*80}")

        return pd.DataFrame(results)

    except Exception as e:
        print(f"🚨 평가 중 심각한 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return None

# ==============================================================================
# 7. 스크립트 실행
# ==============================================================================
if __name__ == '__main__':

    # 평가 실행
    # n_samples: 평가할 샘플 수. 전체(1034개)를 평가하려면 1034로 설정
    evaluation_results_df = run_evaluation_ko_from_csv(
        model,
        tokenizer,
        n_samples=1034, # 테스트를 위해 10개만 실행, 전체는 1034로 변경
        random_seed=15
    )

    # 결과 데이터프레임 출력
    if evaluation_results_df is not None:
        print("\n\n📋 상세 평가 결과 테이블:")
        display(evaluation_results_df)



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Generated SQL: SELECT 
  b.breed_name,
  o.owner_name,
  d.dog_name,
  t.treatment_name,
  p.professional_name
FROM 
  breeds b
  JOIN owners o ON b.breed_id = o.owner_id
  JOIN dogs d ON o.owner_id = d.owner_id
  JOIN treatments t ON d.dog_id = t.dog_id
  JOIN professionals p ON t.treatment_id = p.professional_id;
Exact Match: ❌
Execution: ❌ (Error: no such column: o.owner_name...)
--------------------------------------------------------------------------------

Example 780/1034:
Database: cre_Doc_Template_Mgt
Question (KO): 이름이 'Customer reviews'인 문서의 문단 텍스트는 무엇입니까?
Gold SQL: SELECT T1.paragraph_text FROM Paragraphs AS T1 JOIN Documents AS T2 ON T1.document_id  =  T2.document_id WHERE T2.document_name  =  "Customer reviews"
Generated SQL: SELECT 
  T1.Template_ID,
  T1.Template_Type_Code,
  T1.Template_Type_Description,
  T1.Version_Number,
  T1.Date_Effective_From,
  T1.Date_Effective_To,
  T1.Template_Details,
  T2.Document_ID,
  T2.Document_Name

,index,db_id,question_ko,gold_sql,generated_sql,exact_match,execution_match,execution_error
0,427,museum_visit,2010년 후에 개관한 박물관을 방문하지 않은 방문자 수를 찾으세요.,SELECT count(*) FROM visitor WHERE id NOT IN (...,"SELECT v.Name, v.Num_of_Staff, m.Name, m.Num_o...",False,False,no such column: v.Name
1,23,concert_singer,각 경기장에서 몇 개의 콘서트가 열리나요?,"SELECT T2.name , count(*) FROM concert AS T1 ...","SELECT s.Name, c.concert_Name, c.Year, s.Count...",False,False,no such column: c.Singer_ID
2,74,pets_1,각 종류의 반려동물별 평균 체중은 얼마인가요?,"SELECT avg(weight) , pettype FROM pets GROUP ...","SELECT p.pet_type, AVG(p.weight) \nFROM pets p...",False,False,no such table: students
3,323,cre_Doc_Template_Mgt,템플릿 유형 코드가 PP 또는 PPT인 템플릿의 ID는 무엇입니까?,SELECT template_id FROM Templates WHERE templa...,"SELECT \n T1.Template_Type_Code,\n T1.Templa...",False,False,no such column: T2
4,489,wta_1,각 핸드 타입별 선수 수를 찾으세요.,"SELECT count(*) , hand FROM players GROUP BY ...","SELECT \n p.player_id,\n p.first_name,\n p....",False,False,no such column: p.player_id
...,...,...,...,...,...,...,...,...
1029,727,world_1,아시아나 유럽의 국가들이 차지하는 총 표면적을 알려주세요.,SELECT sum(SurfaceArea) FROM country WHERE Con...,"SELECT c1.Name, c2.Name\nFROM city c1\nJOIN co...",False,False,None
1030,282,employee_hire_evaluation,평가를 한 번도 받지 않은 직원들의 이름은 무엇입니까?,SELECT name FROM employee WHERE Employee_ID NO...,"SELECT e.Name, s.Name\nFROM employee e\nJOIN h...",False,False,None
1031,223,flight_2,어떤 도시가 가장 많은 출발 항공편을 가지고 있습니까?,SELECT T1.City FROM AIRPORTS AS T1 JOIN FLIGHT...,"SELECT a.city, COUNT(f.flight_no) as flight_co...",False,False,no such column: a.city
1032,1013,singer,각 국적별로 그 나라 출신 가수가 몇 명인가요?,"SELECT Citizenship , COUNT(*) FROM singer GRO...","SELECT s.Name, s.Birth_Year, s.Net_Worth_Milli...",False,False,None
